In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#use fastai V2
from fastai.vision.all import *

In [ ]:
lbl_csv = '../input/airbus-ship-detection/train_ship_segmentations_v2.csv'
segmentation_df = pd.read_csv(lbl_csv)
segmentation_df = segmentation_df[segmentation_df['EncodedPixels'].notnull()]
segmentation_df

In [ ]:
#exclude_list = ['6384c3e78.jpg','13703f040.jpg', '14715c06d.jpg',  '33e0ff2d5.jpg',
#                '4d4e09f2a.jpg', '877691df8.jpg', '8b909bb20.jpg', 'a8d99130e.jpg', 
#                'ad55c3143.jpg', 'c8260c541.jpg', 'd6c7f17c7.jpg', 'dc3e7c901.jpg',
#                'e44dffe88.jpg', 'ef87bad36.jpg', 'f083256d8.jpg'] #corrupted images

In [ ]:
#添加一列，保存位置
segmentation_df['filepath']= '../input/airbus-ship-detection/train_v2/'+segmentation_df['ImageId']
#segmentation_df = segmentation_df[:100]#小数据测试
segmentation_df

In [ ]:
#创建语义分割对
os.mkdir('train')
os.mkdir('train/images')
os.mkdir('train/labels')

In [ ]:
# 将rle格式进行解码为图片
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
 
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')

In [ ]:
#数据准备消耗时间约2000s
import cv2
for index in range(len(segmentation_df)):
    ImagePath = segmentation_df.iloc[index]['filepath']
    image = cv2.imread(ImagePath, cv2.IMREAD_UNCHANGED)
    mask = rle_decode(segmentation_df.iloc[index]['EncodedPixels'])
    #写入指定地方
    imgPath = "./train/images/"+segmentation_df.iloc[index]['ImageId']
    mskPath = "./train/labels/"+segmentation_df.iloc[index]['ImageId']+".png"
    cv2.imwrite(imgPath,image)
    cv2.imwrite(mskPath,mask)

In [ ]:
#import  os
#import  zipfile
#startdir = "./train"  #要压缩的文件夹路径，这里选择将input中的所有文件压缩
#file_news = './' +'train.zip' # 压缩后文件夹的名字，这里压缩到kaggle之中的output文件之中，名称为result.zip
#z = zipfile.ZipFile(file_news,'w',zipfile.ZIP_DEFLATED) #参数一：文件夹名
#for dirpath, dirnames, filenames in os.walk(startdir):
#    fpath = dirpath.replace(startdir,'') #这一句很重要，不replace的话，就从根目录开始复制
#    fpath = fpath and fpath + os.sep or ''#实现当前文件夹以及包含的所有文件的压缩
#    for filename in filenames:
#        z.write(os.path.join(dirpath, filename),fpath+filename)
#z.close()
#print ('压缩成功')

In [ ]:
fnames = get_image_files('./train/images/')
lbl_names = get_image_files('./train/labels')

In [ ]:
print (fnames[0],lbl_names[0])#查看train和mask文件细节
get_mask = lambda o:'./train/labels/'+str(o.stem)+'.jpg.png' #路径变化 后缀名变化

In [ ]:
#显示图片对
img_fn = fnames[random.randint(0,len(fnames))]
im = PILImage.create(img_fn)
im.show(figsize=(5,5))
print(im.shape)

In [ ]:
#打印配对的mask
mask_fn = get_mask(img_fn)
msk = PILMask.create(mask_fn)
msk.show(figsize=(5,5), alpha=1)
msk.shape

In [ ]:
#生成DataBlock
binary = DataBlock(blocks=(ImageBlock, MaskBlock( ['Background', 'boat'])),    
                   get_items=get_image_files,    #x的获取方法为get_image_files
                   splitter=RandomSplitter(),    #随机分割
                   get_y=get_mask,              #y的获取方法 尝试get_mask
                   item_tfms=Resize(128,ResizeMethod.Squish),       
                   batch_tfms=[Normalize.from_stats(*imagenet_stats)])  

In [ ]:
#读取图片,并显示样例
dls = binary.dataloaders('./train/images',bs=9)
dls.show_batch( vmin=0, vmax=1)

In [ ]:
learn = unet_learner(dls,resnet34,metrics=Dice).to_fp16()

In [ ]:
learn.fit_flat_cos(10)
learn.recorder.plot_loss()

In [ ]:
learn.save('model1')

In [ ]:
lbl_names = get_image_files('./train/labels')

In [ ]:
#读取sample_submission
submission = pd.read_csv('../input/airbus-ship-detection/sample_submission_v2.csv')
submission

In [ ]:
#验证test目录和csv数据一致
#for idx,name in (enumerate(submission['ImageId'].iloc[:])):
#    name =  '../input/airbus-ship-detection/test_v2/'+ str(name)
#    if(not(os.path.exists(name))):
#        print (idx,name)
#print('done')

In [ ]:
test_csv = '../input/airbus-ship-detection/test_v2/'+submission['ImageId']
test_csv

In [ ]:
def rle_encode(mask):
    pixels = mask.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs

In [ ]:
#输出结果
import time
from datetime import datetime
import pytz

for ibatch in range(156):
    test_csv_part = test_csv[100*ibatch:100*(ibatch+1)]
    test_dl = learn.dls.test_dl(test_csv_part)
    preds=[]
    preds = learn.get_preds(dl=test_dl)
    print(ibatch,time.time())
    for idx in range(100):
        submit_np = np.array(preds[0][idx][0]<0.5).astype(np.uint8) 
        msk = PILMask.create(submit_np)
        msk = msk.resize((768,768),Image.ANTIALIAS)  #中间多倒腾了一次，是因为观察到np的resize导致图像变形
        submit_np2 = np.array(msk)
        rle = rle_encode(submit_np2)
        rle = ' '.join(str(x) for x in rle)
        submission['EncodedPixels'][100*ibatch+idx]=rle

In [ ]:
#tz = pytz.timezone('Asia/Shanghai') #东八区
csv_str = 'submission.csv'
submission.to_csv(csv_str,index=False, header=True)